# AWS 기반 IRT-CAT 시스템 구축 가이드

AWS에서 제시한 아키텍처를 실제로 구현하는 과정을 단계별로 설명합니다. 이 설명에서는 **SageMaker**를 사용하여 모델을 학습하고 배포하며, **API Gateway**, **Lambda**, **RDS**, **S3**, **QuickSight**를 연결하여 IRT 기반 CAT 시스템을 완성하는 과정을 다룹니다.

---

## 1. SageMaker Notebook 인스턴스 생성 및 모델 학습

### 1.1 SageMaker Notebook 인스턴스 생성
- AWS Management Console에서 **SageMaker** 서비스로 이동합니다.
- `Notebook 인스턴스` 메뉴에서 새 인스턴스를 생성하고, 필요한 IAM 역할을 SageMaker에 할당하여 S3에 접근할 수 있도록 설정합니다.
- 인스턴스 설정 완료 후 `시작(Start)`을 클릭하여 Notebook을 실행합니다.

### 1.2 학습 데이터 준비
- S3에 학생 응답 데이터를 업로드합니다. 예를 들어, `responses.csv` 파일 형태로 데이터를 저장할 수 있습니다.
- SageMaker Notebook 인스턴스에서 **boto3** 라이브러리를 사용하여 S3에 저장된 데이터를 Notebook으로 불러옵니다.

### 1.3 모델 학습 코드 실행
- SageMaker Notebook 인스턴스에서 주어진 Python 코드를 실행하여 모델을 학습합니다. 이 코드에서 `em_1PL`, `em_2PL`, `em_3PL` 중 선택하여 데이터를 학습하고, 문항 모수와 학생 능력 수준(θ)을 추정합니다.
- 학습이 완료되면, 학습된 모델 파라미터를 **S3에 저장**하여 이후 모델 배포에 사용할 수 있도록 합니다.

---

## 2. SageMaker 모델을 S3에서 가져와 Endpoint로 배포

### 2.1 모델 아티팩트 저장
- SageMaker Notebook에서 학습이 완료되면 `model.tar.gz` 형식으로 모델 파일을 저장하고 이를 S3 버킷에 업로드합니다.

### 2.2 SageMaker 모델 생성 및 Endpoint 배포
- SageMaker 콘솔에서 `모델` 메뉴로 이동하여 새 모델을 생성합니다.
- S3에서 업로드한 `model.tar.gz` 파일을 모델 소스로 지정합니다.
- 모델 생성 후, `Endpoint 설정` 메뉴에서 해당 모델을 기반으로 엔드포인트를 생성합니다. 이 엔드포인트는 추후 실시간 추론 요청을 처리하는 SageMaker Endpoint로 작동하게 됩니다.

---

## 3. API Gateway 및 Lambda 설정

### 3.1 API Gateway 설정
- AWS Management Console에서 **API Gateway**로 이동하여 새로운 HTTP API를 생성합니다.
- `POST` 메서드를 설정하여 학생의 응답 데이터를 수집할 엔드포인트를 만듭니다.

### 3.2 Lambda 함수 생성
- AWS Lambda로 이동하여 새로운 Lambda 함수를 생성합니다.
- Lambda 함수에서 **SageMaker Endpoint**, **RDS**에 접근할 수 있도록 필요한 IAM 역할을 부여합니다.
- Lambda 코드에 `boto3` 라이브러리를 사용하여 API Gateway로부터 받은 학생의 응답 데이터를 SageMaker Endpoint로 전달하고, 추정된 θ 값을 RDS에 저장하는 로직을 구현합니다.

### 3.3 API Gateway와 Lambda 연결
- API Gateway에서 생성한 `POST` 메서드를 설정하여 Lambda 함수를 트리거하도록 연결합니다.
- 이로써, 학생이 응답을 제출하면 API Gateway를 통해 Lambda가 호출되고, Lambda는 SageMaker Endpoint와 RDS를 통해 실시간 추론과 θ 값 업데이트를 처리하게 됩니다.

---

## 4. RDS 인스턴스 생성 및 데이터베이스 설정

### 4.1 RDS 인스턴스 생성
- AWS RDS 콘솔로 이동하여 MySQL이나 PostgreSQL 인스턴스를 생성합니다.
- VPC 설정에서 Private Subnet에 배치하여 외부 접근을 제한하고, Lambda, SageMaker Endpoint가 접근할 수 있도록 보안 그룹을 설정합니다.
- RDS 인스턴스에 적절한 IAM 역할을 할당하여 Lambda와의 연결을 허용합니다.

### 4.2 데이터베이스 및 테이블 생성
- RDS 인스턴스에 접속하여 `question_bank`, `student_responses`, `theta_values` 등의 테이블을 생성합니다.
- 예를 들어, `theta_values` 테이블은 학생별 θ 값을 저장하고, `student_responses` 테이블은 각 문제에 대한 응답 기록을 저장하도록 구성할 수 있습니다.

### 4.3 Lambda와 RDS 연결 설정
- Lambda 함수에 RDS 데이터베이스 접속 설정을 추가하여, 학생의 θ 값과 응답 데이터를 RDS에 저장하는 기능을 구현합니다.
- 이를 위해 Lambda 코드에서 RDS 데이터베이스 연결 정보를 설정하고, 학생의 θ 값을 저장하거나 업데이트하는 SQL 쿼리를 작성합니다.

---

## 5. 데이터 흐름 및 추론 과정

### 5.1 학생 초기 접속 및 θ 값 설정
- 학생이 첫 접속 시, API Gateway가 Lambda로 요청을 전달하고, Lambda는 초기 θ 값을 설정하여 RDS에 저장합니다.

### 5.2 사전 문제 응답 기록 및 θ 업데이트
- 학생이 문제를 풀고 응답을 제출하면 API Gateway가 Lambda로 응답 데이터를 전달합니다.
- Lambda는 SageMaker Endpoint에 이 데이터를 전송하여 θ 값을 업데이트합니다.
- 업데이트된 θ 값은 다시 Lambda로 반환되고, Lambda는 이를 RDS에 저장하여 다음 문제 선택에 반영합니다.

### 5.3 문제 선택 및 제시
- Lambda는 RDS에서 학생의 현재 θ 값에 따라 적절한 난이도의 문제를 선택하고, 이를 API Gateway를 통해 학생에게 제공합니다.

### 5.4 검사 종료 및 기록 저장
- 검사 종료 기준(예: 특정 문제 수 도달 또는 θ 변화율의 수렴)에 도달하면 Lambda는 최종 결과를 RDS에 기록합니다.

---

## 6. 성능 분석 및 시각화 (QuickSight 사용)

### 6.1 QuickSight와 RDS 연결
- AWS QuickSight 콘솔에서 RDS에 연결하여 데이터 소스를 추가합니다.
- QuickSight에서 RDS에 저장된 θ 값과 응답 기록을 기반으로 시각화 대시보드를 만듭니다.

### 6.2 학습 경향 및 성과 분석
- QuickSight에서 각 학생의 학습 진행 상황과 능력 수준 변화를 시각화하여 CAT 시스템의 성과를 추적합니다.

---

## 최종 아키텍처 플로우 요약

1. **학생이 문제를 풀고 응답을 제출** → API Gateway를 통해 Lambda 호출.
2. **Lambda가 SageMaker Endpoint로 응답 데이터 전송 및 θ 추정 수행.**
3. **SageMaker Endpoint가 업데이트된 θ 값을 반환** → Lambda가 RDS에 θ 값 저장.
4. **Lambda가 θ 값을 기준으로 RDS에서 다음 문제 선택** → API Gateway 통해 학생에게 전송.
5. **검사 종료 후 최종 결과 저장** → QuickSight에서 시각화 대시보드 제공.

이 과정을 통해 AWS에서 IRT 기반 CAT 시스템을 완벽하게 구축할 수 있습니다. **SageMaker Endpoint**는 실시간 추론을 담당하고, **Lambda**는 데이터 관리와 서비스 로직을 조정하며, **QuickSight**는 결과를 분석하고 시각화하는 역할을 합니다.
